Convert Images to a Suitable Color Space: Convert the images to a color space where luminance and chrominance are separated, such as LAB or YCrCb.

In [7]:
import cv2
source_img = cv2.imread('Images/FLUX 1.png')
target_img = cv2.imread('Images/dark_forest.png')

source_lab = cv2.cvtColor(source_img, cv2.COLOR_BGR2LAB)
target_lab = cv2.cvtColor(target_img, cv2.COLOR_BGR2LAB)

**Match the Histogram:** Use a custom function to match the histograms of the source image to the target image. OpenCV doesn't have a direct function for this, so you'll need to manually adjust the color channels.

In [8]:
import numpy as np

def match_histograms(source, template):
    matched = np.zeros_like(source)
    for i in range(3):  # Assuming LAB or YCrCb with 3 channels
        s_values, s_counts = np.unique(source[:, :, i], return_counts=True)
        t_values, t_counts = np.unique(template[:, :, i], return_counts=True)

        s_quantiles = np.cumsum(s_counts).astype(np.float64)
        s_quantiles /= s_quantiles[-1]
        t_quantiles = np.cumsum(t_counts).astype(np.float64)
        t_quantiles /= t_quantiles[-1]

        interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

        matched[:, :, i] = np.interp(source[:, :, i], s_values, interp_t_values)
    return matched

matched_lab = match_histograms(source_lab, target_lab)

**Convert Back to BGR:** After matching the histogram, convert the LAB image back to BGR.

In [9]:
matched_bgr = cv2.cvtColor(matched_lab, cv2.COLOR_LAB2BGR)
cv2.imwrite('darkview.jpg', matched_bgr)

True